In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import prince

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [9]:
df = pd.read_csv("historical-correct-WC.csv")
df.head()

,ID,Date,Venue,T1,T2,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
0,454,22-Oct-22,Sydney,Australia,New Zealand,158,17356,3000.3,22962,1933,...,16350,2714.5,20839,1697,855,21345,859,0.972,928,T2
1,455,23-Oct-22,Melbourne,India,Pakistan,161,18471,3062.5,24674,2166,...,19697,3219.2,24009,2132,745,24144,990,1.380,1070,T1
2,456,25-Oct-22,Perth,Australia,Sri Lanka,159,17459,3020.3,23070,1942,...,17113,2782.2,20204,1828,577,21877,786,0.732,1026,T1
3,457,27-Oct-22,Sydney,South Africa,Bangladesh,145,16281,2716.3,21104,1797,...,11419,1828.0,12794,1167,345,14616,500,0.285,727,T1
4,458,29-Oct-22,Sydney,New Zealand,Sri Lanka,145,16470,2732.0,21034,1712,...,17233,2798.5,20338,1840,579,22028,789,0.724,1032,T1


## data pre-processing

In [10]:
df.shape

(14, 26)

In [11]:
df.dtypes

ID                   int64
Date                object
Venue               object
T1                  object
T2                  object
T1_Mat               int64
T1_BowlsFace         int64
T1_OversBowl       float64
T1_RunsScored        int64
T1_4s                int64
T1_6s                int64
T1_RunsConceded      int64
T1_WktsTaken         int64
T1_W/L             float64
T1_WktsLost          int64
T2_Mat               int64
T2_BowlsFace         int64
T2_OversBowl       float64
T2_RunsScored        int64
T2_4s                int64
T2_6s                int64
T2_RunsConceded      int64
T2_WktsTaken         int64
T2_W/L             float64
T2_WktsLost          int64
Winner              object
dtype: object

In [12]:
#  missing values
df.isnull().sum()

ID                 0
Date               0
Venue              0
T1                 0
T2                 0
T1_Mat             0
T1_BowlsFace       0
T1_OversBowl       0
T1_RunsScored      0
T1_4s              0
T1_6s              0
T1_RunsConceded    0
T1_WktsTaken       0
T1_W/L             0
T1_WktsLost        0
T2_Mat             0
T2_BowlsFace       0
T2_OversBowl       0
T2_RunsScored      0
T2_4s              0
T2_6s              0
T2_RunsConceded    0
T2_WktsTaken       0
T2_W/L             0
T2_WktsLost        0
Winner             0
dtype: int64

In [13]:
# duplicates
df.duplicated().sum()

0

In [14]:
# remove unwanted columns
df1 = df.drop(['Date', 'T1', 'T2'], axis=1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
0,454,Sydney,158,17356,3000.3,22962,1933,892,23142,978,...,16350,2714.5,20839,1697,855,21345,859,0.972,928,T2
1,455,Melbourne,161,18471,3062.5,24674,2166,940,23998,940,...,19697,3219.2,24009,2132,745,24144,990,1.380,1070,T1
2,456,Perth,159,17459,3020.3,23070,1942,897,23342,981,...,17113,2782.2,20204,1828,577,21877,786,0.732,1026,T1
3,457,Sydney,145,16281,2716.3,21104,1797,794,21370,900,...,11419,1828.0,12794,1167,345,14616,500,0.285,727,T1
4,458,Sydney,145,16470,2732.0,21034,1712,863,21455,869,...,17233,2798.5,20338,1840,579,22028,789,0.724,1032,T1


In [15]:
# Convert T1/T2 string columns into int columns of 1/0
df1[['Winner']] = \
(df1[['Winner']] == 'T1').astype(int)
df1.tail()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_BowlsFace,T2_OversBowl,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner
9,463,Sydney,155,17741,2944.4,23730,2176,888,23845,903,...,17460,2838.5,20572,1864,585,22333,802,0.727,1046,1
10,464,Adelaide,171,19938,3253.2,24335,2157,761,24408,1003,...,11614,1868.0,13027,1183,356,14998,510,0.278,743,1
11,465,Sydney,172,20049,3273.2,24452,2164,764,24530,1011,...,16713,2771.2,21343,1732,873,21733,884,0.986,944,1
12,466,Adelaide,164,18835,3118.3,25126,2202,957,24425,958,...,17860,2964.4,23867,2188,890,23983,910,1.260,973,0
13,468,Melbourne,173,20164,3293.2,24595,2178,765,24676,1014,...,17956,2984.4,24033,2201,900,24151,915,1.275,973,0


In [16]:
df1['Winner'].value_counts()

1    10
0     4
Name: Winner, dtype: int64

## feature engineering

In [21]:
# create overs batted var
df1['T1_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6) 
    elif str(round((df1['T1_BowlsFace'][i]/6)-int(df1['T1_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T1_OversBat'][i]= (df1['T1_BowlsFace'][i]/6)
        
df1.head()

df1['T2_OversBat'] = 0
for i in range(0, len(df1)):
    if str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.167' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.333' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.5' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.667' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.833' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6) 
    elif str(round((df1['T2_BowlsFace'][i]/6)-int(df1['T2_BowlsFace'][i]/6),3))[1:] == '.0' :
        df1['T2_OversBat'][i]= (df1['T2_BowlsFace'][i]/6)
        
df1.head(10)

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBowled,T2_OversBowled,T1_OversBat,T2_OversBat
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,855,21345,859,0.972,928,0,3000.500000,2714.833333,2892.666667,2725.000000
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,745,24144,990,1.380,1070,1,3062.833333,3219.333333,3078.500000,3282.833333
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,577,21877,786,0.732,1026,1,3020.500000,2782.333333,2909.833333,2852.166667
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,345,14616,500,0.285,727,1,2716.500000,1828.000000,2713.500000,1903.166667
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,579,22028,789,0.724,1032,1,2732.000000,2798.833333,2745.000000,2872.166667
5,459,Oceania,162,18592,3082.5,24823,2175,946,24151,948,...,805,21466,909,1.246,812,0,3082.833333,2733.000000,3098.666667,2734.166667
6,460,Oceania,154,17620,2924.4,23556,2161,882,23688,897,...,868,21557,879,1.000,938,1,2924.666667,2751.333333,2936.666667,2765.500000
7,461,Oceania,163,18712,3102.3,24948,2184,951,24282,953,...,349,14815,505,0.282,737,1,3102.500000,1848.000000,3118.666667,1919.666667
8,462,Oceania,170,19817,3239.2,24156,2144,751,24300,995,...,810,21592,917,1.261,817,1,3239.333333,2753.000000,3302.833333,2753.833333
9,463,Oceania,155,17741,2944.4,23730,2176,888,23845,903,...,585,22333,802,0.727,1046,1,2944.666667,2838.833333,2956.833333,2910.000000


In [17]:
# create overs bowl var

df1['T1_OversBowled'] = 0

for i in range(0, len(df1)):
    if str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.1' :
        df1['T1_OversBowled'][i]=  int(df1['T1_OversBowl'][i])+ 0.166667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.2' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.333333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.3' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.500000
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.4' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.666667
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.5' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i])+ 0.833333
    elif str(round(df1['T1_OversBowl'][i]-int(df1['T1_OversBowl'][i]),1))[1:] == '.0' :
        df1['T1_OversBowled'][i]= int(df1['T1_OversBowl'][i]) 
 
 
df1['T2_OversBowled'] = 0
    
for i in range(0, len(df1)):
    if str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.1' :
        df1['T2_OversBowled'][i]=  int(df1['T2_OversBowl'][i])+ 0.166667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.2' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.333333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.3' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.500000
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.4' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.666667
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.5' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i])+ 0.833333
    elif str(round(df1['T2_OversBowl'][i]-int(df1['T2_OversBowl'][i]),1))[1:] == '.0' :
        df1['T2_OversBowled'][i]= int(df1['T2_OversBowl'][i]) 
        
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBowled,T2_OversBowled
0,454,Sydney,158,17356,3000.3,22962,1933,892,23142,978,...,20839,1697,855,21345,859,0.972,928,0,3000.500000,2714.833333
1,455,Melbourne,161,18471,3062.5,24674,2166,940,23998,940,...,24009,2132,745,24144,990,1.380,1070,1,3062.833333,3219.333333
2,456,Perth,159,17459,3020.3,23070,1942,897,23342,981,...,20204,1828,577,21877,786,0.732,1026,1,3020.500000,2782.333333
3,457,Sydney,145,16281,2716.3,21104,1797,794,21370,900,...,12794,1167,345,14616,500,0.285,727,1,2716.500000,1828.000000
4,458,Sydney,145,16470,2732.0,21034,1712,863,21455,869,...,20338,1840,579,22028,789,0.724,1032,1,2732.000000,2798.833333


In [18]:
# Venue---> Continent
df['Venue'].unique()

array(['Sydney', 'Melbourne', 'Perth', 'Brisbane', 'Adelaide'],
      dtype=object)

In [19]:
# 1. Venue var
Oceania = ['Sydney', 'Melbourne', 'Auckland', 'Hamilton', 'Brisbane','Wellington', 'Hobart', 'Adelaide', 'Mount Maunganui',
           'Napier', 'Geelong', 'Nelson', 'Carrara', 'Christchurch', 'Canberra', 'Perth', 'Dunedin']

Africa = ['Centurion', 'Johannesburg', 'Cape Town', 'Durban', 'Bloemfontein', 'Potchefstroom', 'Harare', 'East London',
         'Gqeberha', 'Paarl', "St George's"]

Asia = ['Pallekele', 'Colombo (RPS)', 'Hambantota', 'Rajkot', 'Mirpur', 'Dubai (DSC)', 'Chattogram', 'Dharamsala',
        'Cuttack', 'Pune', 'Ranchi', 'Visakhapatnam','Nagpur', 'Eden Gardens', 'Wankhede', 'Bengaluru', 'Mohali',
       'Delhi', 'Abu Dhabi', 'Kanpur', 'Guwahati', 'Lahore', 'Thiruvananthapuram', 'Indore', 'Sylhet', 'Karachi',
       'Lucknow', 'Chennai', 'Hyderabad (Deccan)', 'Ahmedabad', 'Sharjah', 'Jaipur']

Europe = ['The Oval', 'Southampton', 'Chester-le-Street', 'Birmingham', 'Manchester', 'Cardiff', 'Taunton', 'Bristol', 
          'Nottingham', 'Leeds']

America = ['Kingstown', 'Bridgetown', 'Roseau', 'Lauderhill', 'Port of Spain', 'Kingston', 'Basseterre', 'Gros Islet', 
           'Providence', 'Coolidge', 'Tarouba']

for i in range(0, len(df1)):
    if df1['Venue'][i] in Oceania:
        df1['Venue'][i]='Oceania'
    elif df1['Venue'][i]  in Africa:
        df1['Venue'][i] = 'Africa'
    elif df1['Venue'][i]  in Asia:
        df1['Venue'][i] = 'Asia'
    elif df1['Venue'][i]  in Europe:
        df1['Venue'][i] = 'Europe'
    elif df1['Venue'][i]  in America:
        df1['Venue'][i] = 'America'
        
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_RunsScored,T2_4s,T2_6s,T2_RunsConceded,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBowled,T2_OversBowled
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,20839,1697,855,21345,859,0.972,928,0,3000.500000,2714.833333
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,24009,2132,745,24144,990,1.380,1070,1,3062.833333,3219.333333
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,20204,1828,577,21877,786,0.732,1026,1,3020.500000,2782.333333
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,12794,1167,345,14616,500,0.285,727,1,2716.500000,1828.000000
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,20338,1840,579,22028,789,0.724,1032,1,2732.000000,2798.833333


In [22]:
# 2. avg runs scored per over var
df1['T1_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T1_RunsScored/row.T1_OversBat,2) , axis = 1)
df1['T2_AvgRunsScored/Over'] = df1.apply(lambda row: np.round(row.T2_RunsScored/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_WktsTaken,T2_W/L,T2_WktsLost,Winner,T1_OversBowled,T2_OversBowled,T1_OversBat,T2_OversBat,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,859,0.972,928,0,3000.500000,2714.833333,2892.666667,2725.000000,7.94,7.65
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,990,1.380,1070,1,3062.833333,3219.333333,3078.500000,3282.833333,8.01,7.31
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,786,0.732,1026,1,3020.500000,2782.333333,2909.833333,2852.166667,7.93,7.08
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,500,0.285,727,1,2716.500000,1828.000000,2713.500000,1903.166667,7.78,6.72
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,789,0.724,1032,1,2732.000000,2798.833333,2745.000000,2872.166667,7.66,7.08


In [23]:
# 3. avg runs conceded per over var
df1['T1_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T1_RunsConceded/row.T1_OversBowled,2) , axis = 1)
df1['T2_AvgRunsConceded/Over'] = df1.apply(lambda row: np.round(row.T2_RunsConceded/row.T2_OversBowled,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T2_WktsLost,Winner,T1_OversBowled,T2_OversBowled,T1_OversBat,T2_OversBat,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,928,0,3000.500000,2714.833333,2892.666667,2725.000000,7.94,7.65,7.71,7.86
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,1070,1,3062.833333,3219.333333,3078.500000,3282.833333,8.01,7.31,7.84,7.50
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,1026,1,3020.500000,2782.333333,2909.833333,2852.166667,7.93,7.08,7.73,7.86
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,727,1,2716.500000,1828.000000,2713.500000,1903.166667,7.78,6.72,7.87,8.00
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,1032,1,2732.000000,2798.833333,2745.000000,2872.166667,7.66,7.08,7.85,7.87


In [24]:
# 4. create avg boundaries earned per over var
df1['T1_Bound'] = df1.apply(lambda row: np.round((row.T1_4s+row.T1_6s)/row.T1_OversBat,2) , axis = 1)
df1['T2_Bound'] = df1.apply(lambda row: np.round((row.T2_4s+row.T2_6s)/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_OversBowled,T2_OversBowled,T1_OversBat,T2_OversBat,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,3000.500000,2714.833333,2892.666667,2725.000000,7.94,7.65,7.71,7.86,0.98,0.94
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,3062.833333,3219.333333,3078.500000,3282.833333,8.01,7.31,7.84,7.50,1.01,0.88
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,3020.500000,2782.333333,2909.833333,2852.166667,7.93,7.08,7.73,7.86,0.98,0.84
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,2716.500000,1828.000000,2713.500000,1903.166667,7.78,6.72,7.87,8.00,0.95,0.79
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,2732.000000,2798.833333,2745.000000,2872.166667,7.66,7.08,7.85,7.87,0.94,0.84


In [25]:
# 5. avg wickets taken per over
df1['T1_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T1_WktsTaken/row.T1_OversBowled,2) , axis = 1)
df1['T2_Avg_Wkts_Taken'] = df1.apply(lambda row: np.round(row.T2_WktsTaken/row.T2_OversBowled,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_OversBat,T2_OversBat,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,2892.666667,2725.000000,7.94,7.65,7.71,7.86,0.98,0.94,0.33,0.32
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,3078.500000,3282.833333,8.01,7.31,7.84,7.50,1.01,0.88,0.31,0.31
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,2909.833333,2852.166667,7.93,7.08,7.73,7.86,0.98,0.84,0.32,0.28
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,2713.500000,1903.166667,7.78,6.72,7.87,8.00,0.95,0.79,0.33,0.27
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,2745.000000,2872.166667,7.66,7.08,7.85,7.87,0.94,0.84,0.32,0.28


In [26]:
# 6. avg wickets lost
df1['T1_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T1_WktsLost/row.T1_OversBat,2) , axis = 1)
df1['T2_Avg_Wkts_Lost'] = df1.apply(lambda row: np.round(row.T2_WktsLost/row.T2_OversBat,2) , axis = 1)
df1.head()

,ID,Venue,T1_Mat,T1_BowlsFace,T1_OversBowl,T1_RunsScored,T1_4s,T1_6s,T1_RunsConceded,T1_WktsTaken,...,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T2_Avg_Wkts_Lost
0,454,Oceania,158,17356,3000.3,22962,1933,892,23142,978,...,7.94,7.65,7.71,7.86,0.98,0.94,0.33,0.32,0.33,0.34
1,455,Oceania,161,18471,3062.5,24674,2166,940,23998,940,...,8.01,7.31,7.84,7.50,1.01,0.88,0.31,0.31,0.27,0.33
2,456,Oceania,159,17459,3020.3,23070,1942,897,23342,981,...,7.93,7.08,7.73,7.86,0.98,0.84,0.32,0.28,0.33,0.36
3,457,Oceania,145,16281,2716.3,21104,1797,794,21370,900,...,7.78,6.72,7.87,8.00,0.95,0.79,0.33,0.27,0.30,0.38
4,458,Oceania,145,16470,2732.0,21034,1712,863,21455,869,...,7.66,7.08,7.85,7.87,0.94,0.84,0.32,0.28,0.34,0.36


In [27]:
df1.columns

Index(['ID', 'Venue', 'T1_Mat', 'T1_BowlsFace', 'T1_OversBowl',
       'T1_RunsScored', 'T1_4s', 'T1_6s', 'T1_RunsConceded', 'T1_WktsTaken',
       'T1_W/L', 'T1_WktsLost', 'T2_Mat', 'T2_BowlsFace', 'T2_OversBowl',
       'T2_RunsScored', 'T2_4s', 'T2_6s', 'T2_RunsConceded', 'T2_WktsTaken',
       'T2_W/L', 'T2_WktsLost', 'Winner', 'T1_OversBowled', 'T2_OversBowled',
       'T1_OversBat', 'T2_OversBat', 'T1_AvgRunsScored/Over',
       'T2_AvgRunsScored/Over', 'T1_AvgRunsConceded/Over',
       'T2_AvgRunsConceded/Over', 'T1_Bound', 'T2_Bound', 'T1_Avg_Wkts_Taken',
       'T2_Avg_Wkts_Taken', 'T1_Avg_Wkts_Lost', 'T2_Avg_Wkts_Lost'],
      dtype='object')

In [28]:
# remove unvanted columns
data = df1.drop([ 
       'T1_BowlsFace', 'T1_OversBowl',
       'T1_RunsScored', 'T1_4s', 'T1_6s', 'T1_RunsConceded', 'T1_WktsTaken',
       'T1_WktsLost',   'T2_BowlsFace', 'T2_OversBowl',
       'T2_RunsScored', 'T2_4s', 'T2_6s', 'T2_RunsConceded', 'T2_WktsTaken',
       'T2_WktsLost',   'T1_OversBat', 'T2_OversBat',
       'T1_OversBowled', 'T2_OversBowled',  
], axis=1)
data.head()

,ID,Venue,T1_Mat,T1_W/L,T2_Mat,T2_W/L,Winner,T1_AvgRunsScored/Over,T2_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T2_AvgRunsConceded/Over,T1_Bound,T2_Bound,T1_Avg_Wkts_Taken,T2_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T2_Avg_Wkts_Lost
0,454,Oceania,158,1.135,144,0.972,0,7.94,7.65,7.71,7.86,0.98,0.94,0.33,0.32,0.33,0.34
1,455,Oceania,161,1.824,169,1.380,1,8.01,7.31,7.84,7.50,1.01,0.88,0.31,0.31,0.27,0.33
2,456,Oceania,159,1.120,149,0.732,1,7.93,7.08,7.73,7.86,0.98,0.84,0.32,0.28,0.33,0.36
3,457,Oceania,145,1.230,99,0.285,1,7.78,6.72,7.87,8.00,0.95,0.79,0.33,0.27,0.30,0.38
4,458,Oceania,145,0.986,150,0.724,1,7.66,7.08,7.85,7.87,0.94,0.84,0.32,0.28,0.34,0.36


In [29]:
data.shape

(14, 17)

In [30]:
data.columns

Index(['ID', 'Venue', 'T1_Mat', 'T1_W/L', 'T2_Mat', 'T2_W/L', 'Winner',
       'T1_AvgRunsScored/Over', 'T2_AvgRunsScored/Over',
       'T1_AvgRunsConceded/Over', 'T2_AvgRunsConceded/Over', 'T1_Bound',
       'T2_Bound', 'T1_Avg_Wkts_Taken', 'T2_Avg_Wkts_Taken',
       'T1_Avg_Wkts_Lost', 'T2_Avg_Wkts_Lost'],
      dtype='object')

In [31]:
# arrange the columns
data = data.loc[:,['ID',  'Venue', 
             'T1_Mat', 'T1_AvgRunsScored/Over', 'T1_AvgRunsConceded/Over', 'T1_Avg_Wkts_Taken', 'T1_Avg_Wkts_Lost', 'T1_Bound', 'T1_W/L',
             'T2_Mat', 'T2_AvgRunsScored/Over', 'T2_AvgRunsConceded/Over', 'T2_Avg_Wkts_Taken', 'T2_Avg_Wkts_Lost', 'T2_Bound','T2_W/L', 
            'Winner'
           ]]
data.head()

,ID,Venue,T1_Mat,T1_AvgRunsScored/Over,T1_AvgRunsConceded/Over,T1_Avg_Wkts_Taken,T1_Avg_Wkts_Lost,T1_Bound,T1_W/L,T2_Mat,T2_AvgRunsScored/Over,T2_AvgRunsConceded/Over,T2_Avg_Wkts_Taken,T2_Avg_Wkts_Lost,T2_Bound,T2_W/L,Winner
0,454,Oceania,158,7.94,7.71,0.33,0.33,0.98,1.135,144,7.65,7.86,0.32,0.34,0.94,0.972,0
1,455,Oceania,161,8.01,7.84,0.31,0.27,1.01,1.824,169,7.31,7.50,0.31,0.33,0.88,1.380,1
2,456,Oceania,159,7.93,7.73,0.32,0.33,0.98,1.120,149,7.08,7.86,0.28,0.36,0.84,0.732,1
3,457,Oceania,145,7.78,7.87,0.33,0.30,0.95,1.230,99,6.72,8.00,0.27,0.38,0.79,0.285,1
4,458,Oceania,145,7.66,7.85,0.32,0.34,0.94,0.986,150,7.08,7.87,0.28,0.36,0.84,0.724,1


In [32]:
data.shape

(14, 17)

In [33]:
# save the dataset
data.to_csv('historical-featured-WC.csv')